In [196]:
using SwiftObjectStores
using ColoringNames
using Distributions
using MLDataPattern
using Iterators
using MLLabelUtils
using StaticArrays
using Juno
using StatsBase
using Colors
using DataFrames
using Query
using Plots
gr()

Plots.GRBackend()

In [1]:


const od =(MLDataUtils.ObsDim.First(), MLDataUtils.ObsDim.Last())
const serv=SwiftService()

const valid_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/dev.csv")
const valid_hsv, valid_terms_padded, encoding = prepare_data(valid_raw; do_demacate=false)
const valid_text = valid_raw[:, 1]

const train_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/train.csv")
const train_hsv, train_terms_padded, encoding = prepare_data(train_raw, encoding; do_demacate=false)
const train_text = train_raw[:, 1]




INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/MLLabelUtils.ji for module MLLabelUtils.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/MLDataPattern.ji for module MLDataPattern.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/SortingAlgorithms.ji for module SortingAlgorithms.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataFrames.ji for module DataFrames.


1523108-element Array{Any,1}:
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 ⋮           
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      

In [2]:


function pairwise_stats(fun, names, hsvs)
    dt = DataFrame(name=String[], n_samples=Int[], hs=Float64[], hv=Float64[], vs=Float64[])
    @progress for (name, inds) in labelmap(names)
        eg_hsvs = @view hsvs[inds, :]

        hs = fun(eg_hsvs[:,1], eg_hsvs[:,2])
        hv = fun(eg_hsvs[:,1], eg_hsvs[:,3])
        vs = fun(eg_hsvs[:,3], eg_hsvs[:,2])

        push!(dt, [name, length(inds), hs, hv, vs])
    end
    dt
end
spearman = pairwise_stats(corspearman, train_text, train_hsv)



describe(spearman)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

hs
Summary Stats:
Mean:           0.022951
Minimum:        -0.429653
1st Quartile:   -0.086345
Median:         0.019934
3rd Quartile:   0.121130
Maximum:        0.577041
Length:         829
Type:           Float64

hv
Summary Stats:
Mean:           0.000171
Minimum:        -0.577392
1st Quartile:   -0.101860
Median:         0.003642
3rd Quartile:   0.103550
Maximum:        0.497044
Length:         829
Type:           Float64

vs
Summary Stats:
Mean:           -0.048629
Minimum:        -0.497048
1st Quartile:   -0.132881
Median:         -0.050129
3rd Quartile:   0.031004
Maximum:        0.455614
Length:         829
Type:           Float64



In [ ]:
train_cols = hsv2colorant(train_hsv)|>vec

In [ ]:
train_col_hsv = convert(Vector{HSV}, train_cols)
train_col_hsv[1]

In [17]:
T=typeof(train_col_hsv[1])

ColorTypes.HSV{Float32}

In [20]:
function pairwise_fields(T)
    ns = fieldnames(T)
    pairwise_ns = Tuple{Symbol,Symbol}[]
    for (ii, n1) in enumerate(ns)
        for (jj, n2) in enumerate(ns)
            jj<=ii && continue
            push!(pairwise_ns, (n1,n2))
        end
    end
    pairwise_ns
end

pairwise_fields (generic function with 1 method)

In [87]:
dt = DataFrame(
    ;[(:name=> String[]); (:n_samples=> Int[]); 
               [(Symbol(n1,n2)=> Float64[]) for (n1,n2) in pairwise_ns]]...)
dt.columns


Use "Dict(a=>b for (a,b) in c)" instead.


LoadError: AssertionError: frame.inferred

In [90]:
dt = DataFrame([String;Int;[Float64 for _ in pairwise_ns]], 
                [:name;  :n_samples; [Symbol(n1,n2) for (n1,n2) in pairwise_ns]],
                0)
dt.columns

5-element Array{Any,1}:
 String[] 
 Int64[]  
 Float64[]
 Float64[]
 Float64[]

In [92]:
dt = DataFrame(a=Int[])
dt.columns

1-element Array{Any,1}:
 Int64[]

In [235]:
function pairwise_stats{T}(fun, names, data::Vector{T})
    pairwise_ns = pairwise_fields(T)
    dt = DataFrame([String;Int;fill(Float64, 2*length(pairwise_ns))], 
                    [:name;  :n_samples; 
                    [Symbol(n1,n2) for (n1,n2) in pairwise_ns];
                    [Symbol(:abs_,n1,n2) for (n1,n2) in pairwise_ns]],
                    0)
    
    for (name, inds) in labelmap(names)
        sub = @view data[inds]
        row=Dict{Symbol, Any}(:name=>name, :n_samples=>length(inds))
        for (n1,n2) in pairwise_ns
            v1 = getfield.(sub, Scalar(n1))
            v2 = getfield.(sub, Scalar(n2))
            val = fun(v1, v2)
            row[Symbol(n1,n2)] = val
            row[Symbol(:abs_, n1,n2)] = abs(val)
        end
        push!(dt, row)
    end
    dt
end


pairwise_stats (generic function with 2 methods)

In [257]:
:(HSV, RGB, Lab, xyY, X) |> dump

Expr
  head: Symbol tuple
  args: Array{Any}((5,))
    1: Symbol HSV
    2: Symbol RGB
    3: Symbol Lab
    4: Symbol xyY
    5: Symbol X
  typ: Any


In [256]:
for x in :(HSV, RGB, Lab, xyY, X]).args
    @show x
end

x = :HSV
x = :RGB
x = :Lab
x = :xyY
x = :X


In [263]:
Q1(v)=quantile(v,0.25)
Q3(v)=quantile(v,0.75)

Q3 (generic function with 1 method)

In [284]:

space = HSI
space.name.name |> typeof

Symbol

In [285]:
function summarise(funs, names, colors)
    dt = DataFrame(name_stat=Symbol[], f12=Float64[], f13=Float64[], f23=Float64[])
    for space in [RGB, HSV, HSI, HSL, xyY, XYZ, xyY, Lab, Luv, LCHab, LCHuv, DIN99, DIN99d, DIN99o, LMS, YIQ, YCbCr]
        space_name = space.name.name
        
        colors_space = convert(Vector{space}, colors)
        stats = pairwise_stats(corspearman, names, colors_space)
        for fun in funs
            row = Any[Symbol(space_name, :_, fun)]
            for col_ii in [2,1,0]
                column = stats[:, end-col_ii]
                push!(row, fun(column))
            end
            push!(dt, row)
        end
    end
    dt
end

summarise([Q3], train_text, train_cols)

,name_stat,f12,f13,f23
1,RGB_Q3,0.6030468173161735,0.4471926980645492,0.5655902318754856
2,HSV_Q3,0.18613341836244784,0.18665944073636134,0.1627735443944819
3,HSI_Q3,0.2446086788211584,0.23911783105162776,0.6301694579237653
4,HSL_Q3,0.16551731146099502,0.2147074863173264,0.3112894699855042
5,xyY_Q3,0.7230260376798017,0.5024120339688362,0.41654070660522274
6,XYZ_Q3,0.9726011334766465,0.8166502753828205,0.7843590386564617
7,xyY_Q3,0.7230260376798017,0.5024120339688362,0.41654070660522274
8,Lab_Q3,0.5730180530926171,0.4597498889534288,0.6390289755602638
9,Luv_Q3,0.5597935438719273,0.611177475128859,0.4379400128184171
10,LCHab_Q3,0.5258344651603079,0.411030235162374,0.3687582570400751


In [239]:
train_Lab = convert(Vector{Lab}, train_cols)
Lab_stats = pairwise_stats(corspearman, train_text, train_Lab)
describe(Lab_stats)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

la
Summary Stats:
Mean:           -0.281513
Minimum:        -0.921355
1st Quartile:   -0.551229
Median:         -0.336951
3rd Quartile:   -0.081709
Maximum:        0.817061
Length:         829
Type:           Float64

lb
Summary Stats:
Mean:           0.243178
Minimum:        -0.486204
1st Quartile:   0.048399
Median:         0.265873
3rd Quartile:   0.458537
Maximum:        0.904893
Length:         829
Type:           Float64

ab
Summary Stats:
Mean:           -0.362891
Minimum:        -0.979563
1st Quartile:   -0.639029
Median:         -0.367259
3rd Quartile:   -0.156100
Maximum:        0.601683
Length:         829
Type:           Float64

abs_la
Summary Stats:
Mean:      

In [238]:
train_HSV = convert(Vector{HSV}, train_cols)
HSV_stats = pairwise_stats(corspearman, train_text, train_HSV)
describe(HSV_stats)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

hs
Summary Stats:
Mean:           0.017221
Minimum:        -0.429653
1st Quartile:   -0.091609
Median:         0.012737
3rd Quartile:   0.115249
Maximum:        0.577039
Length:         829
Type:           Float64

hv
Summary Stats:
Mean:           -0.002241
Minimum:        -0.577392
1st Quartile:   -0.104592
Median:         0.002000
3rd Quartile:   0.100124
Maximum:        0.497044
Length:         829
Type:           Float64

sv
Summary Stats:
Mean:           -0.048628
Minimum:        -0.497048
1st Quartile:   -0.132840
Median:         -0.050129
3rd Quartile:   0.031004
Maximum:        0.455599
Length:         829
Type:           Float64

abs_hs
Summary Stats:
Mean:        

In [240]:
train_RGB = convert(Vector{RGB}, train_cols)
RGB_stats = pairwise_stats(corspearman, train_text, train_RGB)
summarystats(RGB_stats)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

rg
Summary Stats:
Mean:           0.407982
Minimum:        -0.344066
1st Quartile:   0.226100
Median:         0.377917
3rd Quartile:   0.603047
Maximum:        0.948021
Length:         829
Type:           Float64

rb
Summary Stats:
Mean:           0.315897
Minimum:        -0.283440
1st Quartile:   0.175185
Median:         0.307145
3rd Quartile:   0.447193
Maximum:        0.862251
Length:         829
Type:           Float64

gb
Summary Stats:
Mean:           0.411123
Minimum:        -0.190191
1st Quartile:   0.259813
Median:         0.390688
3rd Quartile:   0.565590
Maximum:        0.900493
Length:         829
Type:           Float64

abs_rg
Summary Stats:
Mean:           0.4

In [241]:
train_XYZ = convert(Vector{XYZ}, train_cols)
XYZ_stats = pairwise_stats(corspearman, train_text, train_XYZ)


name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

xy
Summary Stats:
Mean:           0.939455
Minimum:        0.686607
1st Quartile:   0.914553
Median:         0.949355
3rd Quartile:   0.972601
Maximum:        0.995688
Length:         829
Type:           Float64

xz
Summary Stats:
Mean:           0.725493
Minimum:        0.197383
1st Quartile:   0.650561
Median:         0.742922
3rd Quartile:   0.816650
Maximum:        0.954187
Length:         829
Type:           Float64

yz
Summary Stats:
Mean:           0.677274
Minimum:        0.134936
1st Quartile:   0.590071
Median:         0.707533
3rd Quartile:   0.784359
Maximum:        0.928251
Length:         829
Type:           Float64

abs_xy
Summary Stats:
Mean:           0.9394

In [248]:
train_Luv = convert(Vector{Luv}, train_cols)
Luv_stats = pairwise_stats(corspearman, train_text, train_Luv)
describe(Luv_stats)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

lu
Summary Stats:
Mean:           -0.229521
Minimum:        -0.937783
1st Quartile:   -0.525653
Median:         -0.239831
3rd Quartile:   0.026377
Maximum:        0.828036
Length:         829
Type:           Float64

lv
Summary Stats:
Mean:           0.325315
Minimum:        -0.660747
1st Quartile:   0.084703
Median:         0.375072
3rd Quartile:   0.608203
Maximum:        0.905573
Length:         829
Type:           Float64

uv
Summary Stats:
Mean:           -0.090174
Minimum:        -0.903249
1st Quartile:   -0.351085
Median:         -0.121452
3rd Quartile:   0.142887
Maximum:        0.861654
Length:         829
Type:           Float64

abs_lu
Summary Stats:
Mean:        

In [242]:
train_xyZ = convert(Vector{xyZ}, train_cols)
xyZ_stats = pairwise_stats(corspearman, train_text, train_xyZ)
describe(xyZ_stats)

LoadError: UndefVarError: xyZ not defined

In [224]:
sample = eachbatch(shuffleobs(train_Lab), size=1000) |> first |> collect
xs,ys,zs = obsview(getfield.(sample, fieldnames(Lab)'))

scatter3d(xs, ys, zs; markercolor=sample)

INFO: The specified values for size and/or count will result in 108 unused data points


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,279.187 30.6037,279.187 22.946,283.646 30.6037,279.187 30.6037,266.684 26.7748,268.913 30.6037,266.684 30.6037,254.18 22.946,258.639 30.6037,254.18 
 30.6037,241.677 26.7748,243.906 30.6037,241.677 30.6037,229.173 22.946,233.632 30.6037,229.173 30.6037,216.67 26.7748,218.899 30.6037,216.67 30.6037,204.166 
 22.946,208.625 30.6037,204.166 30.6037,191.663 26.7748,193.892 30.6037,191.663 30.6037,179.159 22.946,183.618 30.6037,179.159 30.6037,166.656 26.7748,168.885 
 30.6037,166.656 30.6037,154.152 22.946,158.611 30.6037,154.152 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 338.069,384.952 355.131,375.017 358.206,376.075 355.131,375.017 388.903,355.354 395.052,357.469 388.903,355.354 422.675,335.691 425.75,336.748 422.675,335.691 
 456.447,316.027 462.596,318.142 456.447,316.027 490.219,296.364 493.293,297.421 490.219,296.364 523.991,276.7 530.14,278.816 523.991,276.7 557.762,257.037 
 560.837,258.095 557.762,257.037 591.534,237.373 597.683,239.489 591.534,237.373 596.063,234.737 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<circle clip-path="url(#clip02)" style="fill:#ffc200; stroke:none; fill-opacity:1" cx="384.979" cy="181.885" r=

In [213]:
zs

10-element Array{Float32,1}:
 -47.7714  
  34.4582  
   0.935067
  58.1518  
 -39.0793  
  12.4516  
  46.126   
  23.4337  
  47.4599  
   6.94147 

In [3]:
histogram(([spearman[:hs],
            spearman[:hv],
            spearman[:vs],
            maximum(abs.(Matrix(spearman[[:hs, :hv, :vs]])), 2)[:]]),
            layout=(4,1),
            legend=false
)

jl_svecref at /home/centos/buildbot/slave/package_tarball64/build/src/julia.h:690 [inlined]
jl_typemap_intersection_array_visitor at /home/centos/buildbot/slave/package_tarball64/build/src/typemap.c:466 [inlined]
jl_typemap_intersection_visitor at /home/centos/buildbot/slave/package_tarball64/build/src/typemap.c:552
ml_matches at /home/centos/buildbot/slave/package_tarball64/build/src/gf.c:2287 [inlined]
jl_matching_methods at /home/centos/buildbot/slave/package_tarball64/build/src/gf.c:2308
_methods_by_ftype at ./reflection.jl:223
unknown function (ip: 0x7f3c2da50cf2)
jl_call_method_internal at /home/centos/buildbot/slave/package_tarball64/build/src/julia_internal.h:210 [inlined]
jl_apply_generic at /home/centos/buildbot/slave/package_tarball64/build/src/gf.c:1950
abstract_call_gf_by_type at ./inference.jl:770
unknown function (ip: 0x7f3c2da53286)
jl_call_method_internal at /home/centos/buildbot/slave/package_tarball64/build/src/julia_internal.h:210 [inlined]
jl_apply_generic at /home

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 -0.2 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60

InterruptException()


In [4]:

high_cor =
    @from r in spearman begin
    @where abs(r.hs) > 0.30 || abs(r.hv) > 0.30 || abs(r.vs) > 0.30
    @orderby abs(r.hs) + abs(r.hv) + abs(r.vs)
    @select r
    @collect DataFrame
end


LoadError: InterruptException: